In [1]:
import pandas as pd 
import numpy as np
import pickle

DATA_PATH = 'data/200/'

In [2]:
with open(f"{DATA_PATH}train_0.pkl", 'rb') as f:
    data = pickle.load(f)

In [3]:
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
from sklearn.metrics import f1_score
from sklearn.preprocessing import LabelEncoder
train, valid = train_test_split(data, test_size=0.2, random_state=42, stratify=data['mbti'], shuffle=True)



y_train = train['mbti']
X_train = train.drop('mbti', axis=1)
y_valid = valid['mbti']
X_valid = valid.drop('mbti', axis=1)

# Convert y_train and y_valid to separate binary columns for each MBTI character
y_train_binary = pd.DataFrame({
    'I-E': y_train.apply(lambda x: 1 if x[0] == 'I' else 0),
    'N-S': y_train.apply(lambda x: 1 if x[1] == 'N' else 0),
    'T-F': y_train.apply(lambda x: 1 if x[2] == 'T' else 0),
    'J-P': y_train.apply(lambda x: 1 if x[3] == 'J' else 0)
})

y_valid_binary = pd.DataFrame({
    'I-E': y_valid.apply(lambda x: 1 if x[0] == 'I' else 0),
    'N-S': y_valid.apply(lambda x: 1 if x[1] == 'N' else 0),
    'T-F': y_valid.apply(lambda x: 1 if x[2] == 'T' else 0),
    'J-P': y_valid.apply(lambda x: 1 if x[3] == 'J' else 0)
})

In [4]:
from catboost import CatBoostClassifier

# List to store models and performance metrics
models = {}
accuracy_scores = {}
f1_scores = {}

# Train and evaluate a model for each binary classification problem
for column in y_train_binary.columns:
    # Initialize the CatBoostClassifier
    model = CatBoostClassifier(verbose=0, task_type="GPU", devices='0:1')
    
    # Fit the model
    model.fit(X_train, y_train_binary[column])
    
    # Predict on the validation set
    y_pred = model.predict(X_valid)
    
    # Calculate accuracy and F1 score
    accuracy = accuracy_score(y_valid_binary[column], y_pred)
    f1 = f1_score(y_valid_binary[column], y_pred)
    
    # Store the model and metrics
    models[column] = model
    accuracy_scores[column] = accuracy
    f1_scores[column] = f1
    
    # Print the results
    print(f"Results for {column}:")
    print(f"Accuracy: {accuracy}")
    print(f"F1 Score: {f1}")
    print("-" * 30)

Results for I-E:
Accuracy: 0.7881019399899555
F1 Score: 0.8814823766890844
------------------------------
Results for N-S:
Accuracy: 0.9246020159076841
F1 Score: 0.9608222146023808
------------------------------
Results for T-F:
Accuracy: 0.608211962298088
F1 Score: 0.6683736771081003
------------------------------
Results for J-P:
Accuracy: 0.6041065651315712
F1 Score: 0.1366935791966992
------------------------------


In [5]:
from xgboost import XGBClassifier

# List to store models and performance metrics
models = {}
accuracy_scores = {}
f1_scores = {}

# Train and evaluate a model for each binary classification problem
for column in y_train_binary.columns:
    # Initialize the CatBoostClassifier
    model = XGBClassifier(device="cuda")
    
    # Fit the model
    model.fit(X_train, y_train_binary[column])
    
    # Predict on the validation set
    y_pred = model.predict(X_valid)
    
    # Calculate accuracy and F1 score
    accuracy = accuracy_score(y_valid_binary[column], y_pred)
    f1 = f1_score(y_valid_binary[column], y_pred)
    
    # Store the model and metrics
    models[column] = model
    accuracy_scores[column] = accuracy
    f1_scores[column] = f1
    
    # Print the results
    print(f"Results for {column}:")
    print(f"Accuracy: {accuracy}")
    print(f"F1 Score: {f1}")
    print("-" * 30)

/home/ubuntu24zmh/MBTI_Classifier/.venv/lib/python3.12/site-packages/xgboost/core.py:158: UserWarning: [10:55:37] WARNING: /workspace/src/common/error_msg.cc:58: Falling back to prediction using DMatrix due to mismatched devices. This might lead to higher memory usage and slower performance. XGBoost is running on: cuda:0, while the input data is on: cpu.
Potential solutions:
- Use a data structure that matches the device ordinal in the booster.
- Set the device for booster before call to inplace_predict.

This warning will only be shown once.

  warnings.warn(smsg, UserWarning)


Results for I-E:
Accuracy: 0.7877048318714304
F1 Score: 0.8811453568777974
------------------------------
Results for N-S:
Accuracy: 0.924561137130777
F1 Score: 0.960797047791306
------------------------------
Results for T-F:
Accuracy: 0.6072775902544996
F1 Score: 0.661205571928764
------------------------------
Results for J-P:
Accuracy: 0.6033707471472454
F1 Score: 0.2558888621075005
------------------------------


In [6]:
from lightgbm import LGBMClassifier 


# List to store models and performance metrics
models = {}
accuracy_scores = {}
f1_scores = {}

# Train and evaluate a model for each binary classification problem
for column in y_train_binary.columns:
    # Initialize the CatBoostClassifier
    model = LGBMClassifier()
        
    # Fit the model
    model.fit(X_train, y_train_binary[column])
    
    # Predict on the validation set
    y_pred = model.predict(X_valid)
    
    # Calculate accuracy and F1 score
    accuracy = accuracy_score(y_valid_binary[column], y_pred)
    f1 = f1_score(y_valid_binary[column], y_pred)
    
    # Store the model and metrics
    models[column] = model
    accuracy_scores[column] = accuracy
    f1_scores[column] = f1
    
    # Print the results
    print(f"Results for {column}:")
    print(f"Accuracy: {accuracy}")
    print(f"F1 Score: {f1}")
    print("-" * 30)

[LightGBM] [Info] Number of positive: 539770, number of negative: 145181
[LightGBM] [Info] Auto-choosing col-wise multi-threading, the overhead of testing was 0.265525 seconds.
You can set `force_col_wise=true` to remove the overhead.
[LightGBM] [Info] Total Bins 51000
[LightGBM] [Info] Number of data points in the train set: 684951, number of used features: 200
[LightGBM] [Info] [binary:BoostFromScore]: pavg=0.788042 -> initscore=1.313162
[LightGBM] [Info] Start training from score 1.313162
Results for I-E:
Accuracy: 0.7881194594657728
F1 Score: 0.8814879175295447
------------------------------
[LightGBM] [Info] Number of positive: 633277, number of negative: 51674
[LightGBM] [Info] Auto-choosing col-wise multi-threading, the overhead of testing was 0.276825 seconds.
You can set `force_col_wise=true` to remove the overhead.
[LightGBM] [Info] Total Bins 51000
[LightGBM] [Info] Number of data points in the train set: 684951, number of used features: 200
[LightGBM] [Info] [binary:BoostFr